# QC-Amp: Colour Factors for Table 1 Diagrams



This notebook uses the `qc_amp` library to compute quantum chromodynamics (QCD) colour factors
for the diagrams listed in an abstracted **Table 1**.

- We first recall the definition of the colour factor and the normalisation.
- We then construct quantum circuits for each diagram using the `qc_amp.circuits` API.
- Finally, we evaluate the corresponding colour factors using `qc_amp.colour_factors`.

## 1. Colour-factor definition and normalisation



We work in an SU($3$) gauge theory with generators $T^a = \lambda^a / 2$ in the fundamental representation, normalised as

$$\operatorname{Tr}(T^a T^b) = \tfrac{1}{2} \, \delta^{ab},$$

so that

$$\operatorname{Tr}(\lambda^a \lambda^b) = 2 \, \delta^{ab}. $$



For a given diagram with $n_q$ external quark lines and $n_g$ gluons, the colour factor is extracted from the quantum circuit as

$$ C = N \, \langle \Omega \mid \psi_\text{final} \rangle, $$

where $\langle \Omega \mid$ is the all-zero bra on all registers, $\mid \psi_\text{final} \rangle$ is the final state produced by the circuit, and

$$ N = N_c^{n_q} (N_c^2 - 1)^{n_g}, \qquad N_c = 3. $$

## 2. Imports and helper functions



We import the QC-Amp library and define a small helper to run a circuit and report the colour factor and normalisation.

In [ ]:
from qc_amp.circuits import quark_emission_absorption

from qc_amp.colour_factors import compute_colour_factor_detailed



def compute_and_report_colour_factor(circuit, n_quarks: int, n_gluons: int, N_c: int = 3):

    """Run the colour-factor computation and return a small dict summary."""

    C, amp_omega, N = compute_colour_factor_detailed(

        circuit,

        n_quarks=n_quarks,

        n_gluons=n_gluons,

        N_c=N_c,

    )

    return {

        'colour_factor': C,

        'overlap_omega': amp_omega,

        'normalisation_N': N,

    }



N_c = 3

## 3. Diagram D1: quark emission and absorption of a gluon



The first Table 1 diagram represents a single quark line emitting and reabsorbing a gluon.
In the colour-algebra language, its colour factor is

$$ C_{\text{D1}} = \sum_{a} \operatorname{Tr}\bigl[(T^a)^2\bigr] = 4, $$

for SU($3$) with $T^a = \lambda^a / 2$.

We now reproduce this result using the quantum circuit implementation.

In [ ]:
# Build the circuit for D1: quark emission and absorption

circ_D1 = quark_emission_absorption(n_vertices=2)

circ_D1.draw(output='mpl')

In [ ]:
# Compute the colour factor for D1

result_D1 = compute_and_report_colour_factor(

    circ_D1,

    n_quarks=1,

    n_gluons=1,

    N_c=N_c,

)

result_D1